In [7]:
%reload_ext autoreload
%autoreload 2

import sys
import os

path_to_project = os.path.abspath(os.path.join(os.getcwd(), '../'))    
sys.path.insert(1, os.path.join(path_to_project))

In [25]:
import itertools
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from src.utils import one_hot
from src.directory import data_dir, NHANES_preprocessed_filename
from src.estimators import aipw_estimator, unadjusted_DM_estimator, ipw_estimator, t_learner, s_learner, x_learner

In [45]:
# load df
NHANES_preprocessed_filepath = os.path.join(data_dir, NHANES_preprocessed_filename)
df = pd.read_csv(NHANES_preprocessed_filepath, index_col='SEQN')

In [46]:
df.dropna(how='any', inplace=True)
df = df[df['age']>=35]
df.rename(columns={'martial_status': 'marital_status'}, inplace=True)
print('# patients with complete data: ', len(df))

# patients with complete data:  2548


In [47]:
# define relevant features
# features
z_col = 'ambient_light'
t_col = 'sleep_deprivation'
y_cols = ['HTN', 'DBP', 'SBP']

# column types
categorical_cols = ['physical_activity', 'ANTIDEPRESSANTS_ANXIOLYTICS', 'GLUCOCORTICOIDS', 'sleep_troubles',
       'sleep_deprivation', 'diabetes', 'smoker', 'race_ethnicity', 'gender', 'health_insurance', 'marital_status','CVD', 
       'smoker_hx', 'HTN']
numerical_cols = ['daily_sedentary', 'accelerometer', 'BMI', 'age', 'poverty_ratio', 'depression', 'yearly_alcohol', 'DBP', 'SBP']

# columns not to transform
all_cols = df.columns
untransformed_cols = [x for x in all_cols if x in [*y_cols, t_col, z_col]]

In [48]:
# prune df
#df.dropna(how='any', inplace=True)

## transform df
# apply scalers
scaler = StandardScaler()
numerical_transformation_cols = list(set(numerical_cols) - set(untransformed_cols))
df[numerical_transformation_cols] = scaler.fit_transform(df[numerical_transformation_cols])

# make z col binary (indicator of recommended max lux value during sleep)
light_cutoff = 1 # nightly minute-mean summed light exposure
df[z_col] = df[z_col].apply(lambda x: 1 if x <= light_cutoff else x)
df[z_col] = df[z_col].apply(lambda x: 0 if x > light_cutoff else x)

# one-hot encode multiclass categoricals
multiclass_cols = df[categorical_cols].columns[df[categorical_cols].nunique() > 2].tolist() 
categorical_transformation_cols = list(set(multiclass_cols) - set(untransformed_cols))

df = one_hot(df, categorical_transformation_cols)
df.columns = df.columns.str.replace('.0', '')

# get df as float
df = df.astype(float)

# get covariates
x_cols = list(set(df.columns) - set([t_col, *y_cols, z_col]))

# update lists of variable type
all_cols = df.columns
categorical_cols = [x for x in df.columns if any([x.startswith(y) for y in categorical_cols])]

In [49]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import statsmodels.api as sm

class PropensityNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PropensityNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, output_dim),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.model(x)

def train_propensity_nn(X, T, n_classes, epochs=100, lr=0.001):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
    T_tensor = torch.tensor(T.values, dtype=torch.long)

    model = PropensityNet(X.shape[1], n_classes)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for _ in range(epochs):
        model.train()
        optimizer.zero_grad()
        logits = model(X_tensor)
        loss = criterion(logits, T_tensor)
        loss.backward()
        optimizer.step()

    with torch.no_grad():
        pi = model(X_tensor).numpy()
    return pi, model, scaler


class OutcomeNet(nn.Module):
    def __init__(self, input_dim):
        super(OutcomeNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

def train_outcome_nn(X, T, Y, n_classes, epochs=100, lr=0.001):
    T_onehot = np.eye(n_classes)[T]
    XT = np.concatenate([X, T_onehot], axis=1)

    scaler = StandardScaler()
    XT_scaled = scaler.fit_transform(XT)

    X_tensor = torch.tensor(XT_scaled, dtype=torch.float32)
    Y_tensor = torch.tensor(Y.values, dtype=torch.float32).unsqueeze(1)

    model = OutcomeNet(XT.shape[1])
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    for _ in range(epochs):
        model.train()
        optimizer.zero_grad()
        y_pred = model(X_tensor)
        loss = criterion(y_pred, Y_tensor)
        loss.backward()
        optimizer.step()

    return model, scaler


def aipw_estimate(X, T, Y, pi, mu_model, scaler, n_classes):
    n = len(X)
    tau_i = {}
    mu = np.zeros((n, n_classes))

    for k in range(n_classes):
        T_k = np.eye(n_classes)[[k] * n]
        XT_k = np.concatenate([X, T_k], axis=1)
        XT_k_scaled = scaler.transform(XT_k)
        X_tensor = torch.tensor(XT_k_scaled, dtype=torch.float32)

        with torch.no_grad():
            preds = mu_model(X_tensor).numpy().flatten()

        mu[:, k] = preds
        treated = (T == k).astype(float).values
        ipw_term = treated / pi[:, k] * (Y.values - mu[:, k])
        tau_i[k] = mu[:, k] + ipw_term

    results = {}
    for i in range(n_classes):
        for j in range(i + 1, n_classes):
            diff = tau_i[j] - tau_i[i]
            results[f"{treatment_classes[j]} vs {treatment_classes[i]}"] = {
                "tau": diff.mean(),
                "variance": diff.var()
            }

    return results, mu


def run_weighted_logistic_regression(df, treatment_col, outcome_col, pi, treatment_classes):
    # Use encoded treatment as input
    T = df['T_encoded']
    Y = df[outcome_col]

    # Dummy encode treatment (reference = first class)
    T_dummies = pd.get_dummies(T, prefix='T', drop_first=True)
    X = sm.add_constant(T_dummies)
    weights = 1.0 / pi[np.arange(len(pi)), T]

    # Fit weighted logistic regression
    model = sm.Logit(Y, X)
    result = model.fit(weights=weights, disp=False)
    X.columns = ['Intercept'] + [f"{treatment_classes[int(c.split('_')[1])]} vs {treatment_classes[0]}" for c in T_dummies.columns]
    result.model.exog_names = X.columns  # rename for output readability

    return result



In [50]:
# Encode treatment into integers
df['T_encoded'], treatment_classes = pd.factorize(df['sleep_deprivation'])
n_classes = len(treatment_classes)

# Extract input arrays
X = df[x_cols].values
T = df['T_encoded']
Y = df['HTN']

In [54]:
pi, prop_model, X_scaler = train_propensity_nn(X, T, n_classes)
mu_model, XT_scaler = train_outcome_nn(X, T.values, Y, n_classes)

In [14]:
class OutcomeNet(nn.Module):
    def __init__(self, input_dim):
        super(OutcomeNet, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

def train_outcome_nn(X, T, Y, n_classes, epochs=100, lr=0.001):
    # One-hot encode T and concatenate with X
    T_onehot = np.eye(n_classes)[T]
    XT = np.concatenate([X, T_onehot], axis=1)
    
    scaler = StandardScaler()
    XT_scaled = scaler.fit_transform(XT)

    X_tensor = torch.tensor(XT_scaled, dtype=torch.float32)
    Y_tensor = torch.tensor(Y.values, dtype=torch.float32).unsqueeze(1)

    model = OutcomeNet(XT.shape[1])
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        y_pred = model(X_tensor)
        loss = criterion(y_pred, Y_tensor)
        loss.backward()
        optimizer.step()

    return model, scaler


In [15]:
def aipw_estimate(X, T, Y, pi, mu_model, scaler, n_classes):
    n = len(X)
    tau_i = {}
    mu = np.zeros((n, n_classes))

    for k in range(n_classes):
        # Construct counterfactual inputs for each treatment level
        T_k = np.eye(n_classes)[[k] * n]
        XT_k = np.concatenate([X, T_k], axis=1)
        XT_k_scaled = scaler.transform(XT_k)

        X_tensor = torch.tensor(XT_k_scaled, dtype=torch.float32)
        with torch.no_grad():
            preds = mu_model(X_tensor).numpy().flatten()

        mu[:, k] = preds

        # IPW adjustment
        treated = (T == k).astype(float).values
        ipw_term = treated / pi[:, k] * (Y.values - mu[:, k])
        tau_i[k] = mu[:, k] + ipw_term

    # Pairwise ATEs
    results = {}
    for i in range(n_classes):
        for j in range(i + 1, n_classes):
            diff = tau_i[j] - tau_i[i]
            results[f"{j} vs {i}"] = {
                "tau": diff.mean(),
                "variance": diff.var()
            }

    return results, mu


In [16]:
import statsmodels.api as sm

def run_weighted_logistic_regression(data, T_col, Y_col, pi, ref_group=0):
    # Create dummy vars for treatment
    T_dummies = pd.get_dummies(data[T_col], prefix='T', drop_first=True)
    T_names = T_dummies.columns.tolist()
    X = T_dummies
    X = sm.add_constant(X)

    # Inverse probability weights
    weights = 1.0 / pi[np.arange(len(pi)), data[T_col].values]

    # Logistic regression with IPW weights
    model = sm.Logit(data[Y_col], X)
    result = model.fit(weights=weights, disp=False)

    return result


In [17]:
# Assume: `data` is a pandas DataFrame with 'sleep_deprivation', 'hypertension', and covariates
T = df['sleep_deprivation']
Y = df['HTN']
X = df[x_cols].values
n_classes = T.nunique()

In [22]:
pi, prop_model, X_scaler = train_propensity_nn(df[x_cols], T, n_classes)


In [30]:
# Convert treatment labels to integers if not already
T_encoded, T_classes = pd.factorize(df['sleep_deprivation'])
df['T_encoded'] = T_encoded

In [35]:
mu_model, XT_scaler = train_outcome_nn(X, df['T_encoded'].values, Y, n_classes)


In [38]:
results, mu = aipw_estimate(X, T, Y, pi, mu_model, XT_scaler, n_classes)

In [39]:
results

{'1 vs 0': {'tau': -0.05634927771866319, 'variance': 3.6802786884034817},
 '2 vs 0': {'tau': -0.05312525559190594, 'variance': 9.778452056198859},
 '2 vs 1': {'tau': 0.0032240221267572137, 'variance': 6.693953511639587}}

In [43]:
logit_result = run_weighted_logistic_regression(df, 'T_encoded', 'HTN', pi)

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [ ]:
# Assume: `data` is a pandas DataFrame with 'sleep_deprivation', 'hypertension', and covariates
T = df['sleep_deprivation']
Y = df['HTN']
X = df[x_cols].values
n_classes = T.nunique()

# Step 1: Train propensity model
pi, prop_model, X_scaler = train_propensity_nn(df[x_cols], T, n_classes)

# Step 2: Train outcome model

# Convert treatment labels to integers if not already
T_encoded, T_classes = pd.factorize(df['sleep_deprivation'])
df['T_encoded'] = T_encoded

mu_model, XT_scaler = train_outcome_nn(X, df['T_encoded'].values, Y, n_classes)

# Step 3: Compute AIPW estimates
results, mu = aipw_estimate(X, T, Y, pi, mu_model, XT_scaler, n_classes)

# Step 4: Weighted logistic regression
logit_result = run_weighted_logistic_regression(data, 'sleep_deprivation', 'hypertension', pi)
print(logit_result.summary())


In [120]:
results, preds = aipw_estimator(df, 
                treatment_var=t_col, 
                outcome_var='HTN', 
                covariates=x_cols,
                continuous_outcome=False)

	S-learner score: 0.7884615384615384
Feature Importance:  [1.49980180e-01 9.83740879e-03 7.73640218e-05 3.58134957e-03
 4.62744444e-02 0.00000000e+00 7.57335057e-02 1.49470349e-03
 6.91042464e-04 2.10812783e-03 0.00000000e+00 8.06301651e-04
 3.08302280e-03 0.00000000e+00 8.68250147e-03 8.39488137e-04
 2.74526261e-03 2.77988864e-03 5.18073429e-03 6.24120641e-04
 1.90852194e-02 6.68592874e-04 0.00000000e+00 4.81749192e-04
 8.51365573e-02 4.05790310e-02 4.75000392e-04 2.67353716e-02
 1.98561028e-02 1.94659150e-03 1.33971060e-02 4.23662637e-01
 3.19564570e-03 0.00000000e+00 2.05318625e-02 9.32497513e-03
 3.19595378e-03 1.54241578e-03 5.98022938e-03 2.05386366e-03
 7.63164900e-03]


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- _
- a
- d
- e
- i
- ...


In [8]:
estimators = [unadjusted_DM_estimator, ipw_estimator, t_learner, s_learner, x_learner, aipw_estimator]
estimator_names = estimator_names = [x.__name__ for x in estimators]
pairs = [' vs '.join(x[::-1]) for x in itertools.combinations(df[t_col].unique().astype(int).astype(str), 2)]
index = pd.MultiIndex.from_product([estimator_names, pairs, y_cols], names=['estimator', 'pair','outcome'])
tau_results = pd.DataFrame(index=index, columns=['tau'])

In [15]:
results

{'1 vs 0': {'tau': -0.027609923703864274, 'variance': 0.6068964341725066},
 '2 vs 0': {'tau': -0.015327715135803135, 'variance': 1.0969490768404648},
 '2 vs 1': {'tau': 0.01228220856806113, 'variance': 0.974821312483853}}

In [13]:
preds

array([[0.79683972, 0.7463666 , 0.7463666 ],
       [0.89624143, 0.89624143, 0.89624143],
       [0.70306437, 0.70306437, 0.70306437],
       ...,
       [0.40388258, 0.40388258, 0.40388258],
       [0.85581417, 0.85581417, 0.85581417],
       [0.79797705, 0.79797705, 0.79797705]])

In [127]:
for tau_estimator, outcome in itertools.product(estimators, y_cols):
    estimator_name = tau_estimator.__name__
    print(f'Estimating effect of {t_col} on {outcome} using {estimator_name}...')
    continuous_outcome = False if outcome == 'HTN' else True
    results = tau_estimator(df, 
                            treatment_var=t_col, 
                            outcome_var=outcome, 
                            covariates=x_cols,
                            continuous_outcome=continuous_outcome)
    
    for pair in results.keys():
        tau = results[pair]['tau']
        tau_results.loc[(estimator_name, pair, outcome)] = tau

Estimating effect of sleep_deprivation on HTN using unadjusted_DM_estimator...
Estimating effect of sleep_deprivation on DBP using unadjusted_DM_estimator...
Estimating effect of sleep_deprivation on SBP using unadjusted_DM_estimator...
Estimating effect of sleep_deprivation on HTN using ipw_estimator...
Estimating effect of sleep_deprivation on DBP using ipw_estimator...
Estimating effect of sleep_deprivation on SBP using ipw_estimator...
Estimating effect of sleep_deprivation on HTN using t_learner...
Estimating effect of sleep_deprivation on DBP using t_learner...
Estimating effect of sleep_deprivation on SBP using t_learner...
Estimating effect of sleep_deprivation on HTN using s_learner...
Estimating effect of sleep_deprivation on DBP using s_learner...
Estimating effect of sleep_deprivation on SBP using s_learner...
Estimating effect of sleep_deprivation on HTN using x_learner...
Estimating effect of sleep_deprivation on DBP using x_learner...
Estimating effect of sleep_deprivati

In [129]:
tau_results.reset_index(inplace=True)

In [131]:
tau_results[tau_results['outcome']=='HTN']

,estimator,pair,outcome,tau
0,unadjusted_DM_estimator,1 vs 0,HTN,-0.078861
3,unadjusted_DM_estimator,2 vs 0,HTN,-0.015747
6,unadjusted_DM_estimator,2 vs 1,HTN,0.063114
9,ipw_estimator,1 vs 0,HTN,0.005025
12,ipw_estimator,2 vs 0,HTN,-0.139962
15,ipw_estimator,2 vs 1,HTN,-0.144987
18,t_learner,1 vs 0,HTN,0.0
21,t_learner,2 vs 0,HTN,0.0
24,t_learner,2 vs 1,HTN,0.0
27,s_learner,1 vs 0,HTN,-0.010694
